## Ogłoszenie

- za tydzień 8 czerwca zajęcia z powodów ode mnie niezależnych  odbędą się krócej (do 12:30). W związku z tym zachęcam do przyjścia już o 11:00, a nie o 11:15, będę w sali mogła odebrać zadania.

- w następny piątek, 9 czerwca w godzina wykładu (9:35-11:05) odbędzie się dodatkowe zaliczenie czyli możliwość oddania zadań, dla tych, którzy nie zdążyli.

- UWAGA: zarezerwowałam na tę okoliczność salę z komputerami 4.31 i tam będę czekać o 9:35 (nie na sali wykładowej). Jeśli nikt nie przyjdzie, będę w tych godzinach w swoim pokoju 3.54 - można przyjść i oddać.

- po terminie 9 czerwca nie będzie już możliwości oddania zadań, wpisuję wszystko do WU.

## Plan wykładu

+ Uruchamianie programów zewnętrznych
+ Wywoływanie C
+ Zanurzanie Julii w innych programach
+ Tworzenie własnych pakietów
+ JuliaCon

## Uruchamianie programów zewnętrznych

In [2]:
cmd =`python -c "print 'dwa plus dwa to {}'.format(2+2)"`

`python -c "print 'dwa plus dwa to {}'.format(2+2)"`

- Zamiast natychmiast uruchamiać polecenie, Julia tworzy obiekt Cmd do reprezentowania polecenia.
- Można użyć tego obiektu do połączenia polecenia z innymi (pipe), uruchomienia itp.
- Gdy polecenie jest uruchamiane,  wyjście domyślnie przechodzi do STDOUT
- Polecenie nie jest uruchamiane przez powłokę,  Julia analizuje składnię poleceń bezpośrednio, odpowiednio interpolując zmienne 
- Polecenie jest uruchamiane jako proces potomny, używając wywołań fork i exec.

In [3]:
typeof(cmd)

Cmd

In [4]:
run(cmd)

dwa plus dwa to 4


In [5]:
# czytanie outputu 
a = readstring(cmd)

"dwa plus dwa to 4\n"

In [6]:
# obcięcie końca linii.
chomp(a)

"dwa plus dwa to 4"

In [72]:
# open() służy to komunikacji z wejściem/wyjściem danej komendy  
open(`less`, "w", STDOUT) do io
           for i = 1:3
               println(io, i)
           end
       end

1
2
3


### Interpolacja

In [8]:
file = "/etc/passwd"

"/etc/passwd"

In [9]:
# interpolacja
cmd = `sort $file`

`sort /etc/passwd`

In [7]:
file1 = "/Volumes/External HD/data.csv"

"/Volumes/External HD/data.csv"

In [9]:
# Julia wstawia znaki '' , aby pokazać użytkownikowi, że zadbała o spację w nazwie pliku. 
`sort $file1`

`sort '/Volumes/External HD/data.csv'`

In [10]:
# powyższe działa też w przypadku iterpolacji
path = "/Volumes/External HD"
name = "data"
ext = "csv"
`sort $path/$name.$ext`

`sort '/Volumes/External HD/data.csv'`

In [11]:
#jeśli chcemy więcej plików - robimy tablicę
files = ["/etc/passwd","/Volumes/External HD/data.csv"]
`grep foo $files`

`grep foo /etc/passwd '/Volumes/External HD/data.csv'`

In [11]:
names = ["foo","bar","baz"]

3-element Array{ASCIIString,1}:
 "foo"
 "bar"
 "baz"

In [12]:
exts = ["aux","log"]

2-element Array{ASCIIString,1}:
 "aux"
 "log"

In [13]:
# iloczyn kartezjański zbioru names i exts
`ls -f $names.$exts`

`ls -f foo.aux foo.log bar.aux bar.log baz.aux baz.log`

In [14]:
# użycie pipe 
run(pipeline(`cut -d: -f3 /etc/passwd`, `sort -n`, `tail -n5`))

115
116
1000
1001
65534


## Wywoływanie funkcji  C w Julii

Julia ma filozofię "no boilerplate": 

- funkcje mogą być wywoływane bezpośrednio z Julii bez jakiegokolwiek kodu "kleju", generowania kodu lub kompilacji - nawet  w trybie interaktywnym.

- Osiąga się to poprzez odpowiednie wywołanie z użyciem składni ccall (), która wygląda jak zwykłe wywołanie funkcji.

- Kod, który ma zostać wywołany, musi być dostępny jako biblioteka współdzielona ( np. opcja -shared  -fPIC) 

- Instrukcje maszyn generowane przez kompilator JIT  są takie same, jak natywne wywołanie C (narzut jest taki sam)   

- Biblioteki i funkcje współdzielone są odwoływane przez krotkę formularza (:function, "library") lub ("function", "library"), gdzie funkcja jest nazwą wywoływaną przez C. 

- Nazwa funkcji może być użyta samodzielnie w miejsce krotki (:funkcja lub "funkcja"). W takim przypadku nazwa funkcji bedzie wzięta z  bieżącego procesu.

- można też  użyć ccall(), aby faktycznie wygenerować wywołanie funkcji biblioteki

In [13]:
#(:Funkcja, biblioteka), typ zwracany, krotka typów wejściowych , argumenty
t = ccall( (:clock, "libc"), Int32, ())

138228795

In [14]:
typeof(t)

Int32

In [24]:
# musi być  (Cstring,), a nie  (Cstring) żeby uzyskać krotkę
shell = ccall((:getenv, "libc"), Cstring, (Cstring,), "SHELL")

Cstring(0x00007ffc470d5a93)

In [27]:
typeof(shell)

Cstring

In [25]:
# konwersja na Juliowy String. unsafe t.j. zakończy się błędem, jeśli wskaźnik nie wskazuje na co powinien . 
shell_in_julia=unsafe_string(shell)

"/bin/bash"

In [26]:
typeof(shell_in_julia)

String

In [17]:
# zwykle opakowuje się ccall tak, aby obsłużyc błedy.
# Błedy w różnych bibliotekach C są zwracane w różny sposób -może być NULL, może byc -1 itd.
function getenv(var::AbstractString)
  val = ccall((:getenv, "libc"),
              Cstring, (Cstring,), var)
  if val == C_NULL
    error("getenv: undefined variable: ", var)
  end
  unsafe_string(val)
end


getenv (generic function with 1 method)

In [20]:
getenv("PATH")

"/home/user/.julia/v0.5/Conda/deps/usr/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin"

In [21]:
getenv("FOOBAR")

LoadError: getenv: undefined variable: FOOBAR

## Zanurzanie w C - przykład

```c
#include <julia.h>

int main(int argc, char *argv[])
{
    /* required: setup the Julia context */
    jl_init(NULL);

    /* run Julia commands */
    jl_eval_string("print(sqrt(2.0))");

    /* strongly recommended: notify Julia that the
         program is about to terminate. this allows
         Julia time to cleanup pending write requests
         and run all finalizers
    */
    jl_atexit_hook(0);
    return 0;
}
}
```
---

```
gcc -o test -fPIC -I$JULIA_DIR/include/julia -L$JULIA_DIR/lib test.c -ljulia $JULIA_DIR/lib/julia/libstdc++.so.6

```
---



## Wywoływanie kodu z Pythona
```julia
pkg.add("PyCall")
```

In [41]:
using PyCall

In [33]:
@pyimport numpy as np

In [34]:
a = np.array([1,2,3,4])

4-element Array{Int64,1}:
 1
 2
 3
 4

In [61]:
typeof(a)

Array{Int64,1}

In [62]:
np.sin(a)

4-element Array{Float64,1}:
  0.841471
  0.909297
  0.14112 
 -0.756802

In [43]:
pyeval("[i**2 for i in range(10)]")

10-element Array{Any,1}:
  0
  1
  4
  9
 16
 25
 36
 49
 64
 81

In [35]:
@pyimport os

In [36]:
# różnica  wołamy foo[:bar] and foo[:bar](...) a nie foo.bar and foo.bar(...)
os.sys.path

LoadError: type PyObject has no field path

In [37]:
os.sys[:path]

7-element Array{Any,1}:
 "/usr/lib/python2.7"                      
 "/usr/lib/python2.7/plat-x86_64-linux-gnu"
 "/usr/lib/python2.7/lib-tk"               
 "/usr/lib/python2.7/lib-old"              
 "/usr/lib/python2.7/lib-dynload"          
 "/usr/local/lib/python2.7/dist-packages"  
 "/usr/lib/python2.7/dist-packages"        

## Pakiety

- Pakiety Julia to po prostu repozytoria git

- Oficjalne pakiety Julia są zarejestrowane w repozytorium METADATA.jl, dostępne w dobrze znanej lokalizacji
https://github.com/JuliaLang/METADATA.jl

- użytkownicy mogą tworzyć swoje własne repozytoria METADATA. Pozwala to na wybieranie, które są dostępne do automatycznej instalacji. Można dopuścić tylko sprawdzone i zatwierdzone wersje pakietów albo udostępniać prywatne pakiety

- Polecenia Pkg.add() i Pkg.rm() współdziałają z zarejestrowanymi pakietami

- menedżer pakietów może również instalować i pracować z niezarejestrowanymi pakietami. Aby zainstalować niezarejestrowany pakiet, używa się Pkg.clone(URL), gdzie URL to git URL, z którego pakiet można sklonować.

```julia
Pkg.clone("git://example.com/path/to/Package.jl.git")
```

- każdy użytkownik ma lokane METADATA w katalogu .julia/VERSION/METADATA np. /home/user/.julia/v0.5/METADATA

- Pakiet do tworzenia i rejestrowanie pakietów

```julia
Pkg.add("PkgDev")
```

In [75]:
# tworzymy pakiet lokalnie 
# jeśli mamy skonfigurowany dostęp do githuba poprzez gita 
# (https://help.github.com/articles/connecting-to-github-with-ssh/) PkgDev wygeneruje dla nas url
# nie wygeneruje zdalnego repo  na Githubie (trzeba to zrobić ręcznie)

using PkgDev
PkgDev.generate("AGH_Julia_Lab2","MIT", force=true)

INFO: Initializing AGH_Julia_Lab2 repo: /home/user/.julia/v0.5/AGH_Julia_Lab2
INFO: Origin: https://github.com/kzajac/AGH_Julia_Lab2.jl.git
INFO: Generating LICENSE.md
INFO: Generating README.md
INFO: Generating src/AGH_Julia_Lab2.jl
INFO: Generating test/runtests.jl
INFO: Generating REQUIRE
INFO: Generating .gitignore
INFO: Generating .travis.yml
INFO: Generating appveyor.yml
INFO: Generating .codecov.yml
INFO: Committing AGH_Julia_Lab2 generated files


In [76]:
# po zakomitowaniu zmian możemy zarejestrować nasz pakiet w lokalnym METADATA
PkgDev.register("AGH_Julia_Lab2")

INFO: Registering AGH_Julia_Lab2 at https://github.com/kzajac/AGH_Julia_Lab2.jl.git
INFO: Committing METADATA for AGH_Julia_Lab2


In [77]:
# można otagować numerem wersji
PkgDev.tag("AGH_Julia_Lab2")

INFO: Tagging AGH_Julia_Lab2 v0.0.1
INFO: Committing METADATA for AGH_Julia_Lab2


Jeśli chcemy, aby nasz pakiet stał się oficjalnym pakietem należy zarejestrować go w oficjalnym repo METADATA poprzez wykonanie kroków:

- zrobic fork repozytorium METADATA  na własne konto na Githubie 
- zrobić operacje push do tego "zforkowanego" repozytorium  
- otworzyć  a pull request

Można to zrobić ręcznie albo  komendą  
```julia
PkgDev.publish()
``` 


## JuliaCon
http://juliacon.org/2017/

- Celeste - analiza obrazów z ciałami niebieskimi

- ciała niebieskie dostarczają fotonów, które składają się na wyprodukowanie pixela w obrazku

- duże dane (teleskopy produkują setki GB  na dobę, będą produkować TB )

- https://github.com/jeff-regier/Celeste.jl